In [1]:
import psi4
import numpy as np
from P4toC4_aux import *

In [2]:
#  C4: STO-3G  -74.963 319 056 386 7

psi4.set_memory('500 MB')
psi4.core.set_global_option("BASIS", "cc-pVDZ")
psi4.core.set_global_option("SCF_TYPE", "pk")
psi4.core.set_global_option("PUREAM", "True")
psi4.core.set_output_file('output.dat', False)

# O
# H 1 0.96
# H 1 0.96 2 104.5


h2o = psi4.geometry("""
0 1
O            0.000000000000     0.000000000000    -0.065775570547
H            0.000000000000    -0.759061990794     0.521953018286
H            0.000000000000     0.759061990794     0.521953018286
symmetry c1
""")

E, wf = psi4.energy('scf', return_wfn=True)
E

-76.02665366131475

In [3]:
mapping, scale =basis_mapping(wf.basisset(), verbose=0)
mapping

array([ 0,  1,  2,  7,  3,  5,  8,  4,  6,  9, 11, 13, 12, 10, 14, 15, 18,
       16, 17, 19, 20, 23, 21, 22])

In [4]:
# basis=wf.basisset()
# print(basis.genbas())
# basis.output('')

In [5]:
n_el = wf.nalpha()
eps = np.array(wf.epsilon_a())
for i in range(n_el-3,n_el+2):
    print(f' {i+1:3d}  {eps[i]:12.7f} {eps[i]*27.21138:10.6f} eV')

   3    -0.6977965 -18.988007 eV
   4    -0.5660912 -15.404123 eV
   5    -0.4929537 -13.413952 eV
   6     0.1851020   5.036880 eV
   7     0.2558498   6.962026 eV


In [6]:
# Psi4
C=np.array(wf.Ca())
print('Psi4 MOs:', wf.nmo())
# Cfour
norb, cs = read_oldmos('OLDMOS.H2O.PVDZ', verbose=1)

Psi4 MOs: 24
reading orbitals from OLDMOS.H2O.PVDZ
  144 lines, 4 orbitals at the bottom
  nAOs = nMOs = 24


In [7]:
mo1,mo2=1,5 # valence
ao1,ao2=0,3 # s
print(np.round( C[ao1:ao2,mo1:mo2],4))
print(np.round(cs[ao1:ao2,mo1:mo2],4))

[[ 0.0081 -0.      0.0022  0.    ]
 [-0.4432 -0.     -0.1502  0.    ]
 [-0.3776  0.     -0.3489 -0.    ]]
[[ 0.0081  0.      0.0022 -0.    ]
 [-0.4432  0.     -0.1502  0.    ]
 [-0.3776 -0.     -0.3489  0.    ]]


In [7]:
mo=5
ao1,ao2=3,9  # O-p for PVDZ
ao1,ao2=9,14 # O-d for PVDZ
print('    Cfour        Psi4')
for ao in range(ao1,ao2):
    print(f'{ao:3d} {cs[ao,mo]:10.6f} {C[ao,mo]:10.6f}')

    Cfour        Psi4
  9  -0.002174  -0.007531
 10  -0.000000  -0.000000
 11  -0.000000   0.000000
 12   0.004922   0.009844
 13  -0.000000   0.000000


In [8]:
C2 = psi4_to_c4(C,mapping, scale)

In [9]:
mo=5
ao1,ao2=3,9  # O-p for PVDZ
ao1,ao2=9,14 # O-d for PVDZ
#ao1,ao2=3,14 # O p and d
print('    Cfour        Psi4')
for ao in range(ao1,ao2):
    print(f'{ao:3d} {cs[ao,mo]:10.6f} {C2[ao,mo]:10.6f}')

    Cfour        Psi4
  9  -0.002174  -0.002174
 10  -0.000000   0.000000
 11  -0.000000  -0.000000
 12   0.004922   0.004922
 13  -0.000000   0.000000


In [13]:
np.max(cs-C2)

4.0489283886745164e-06

In [10]:
write_oldmos('PSIMOS', C2)